In [2]:
import pandas as pd
import os

# Function to identify date format
def identify_date_format(df):
    for col in df.columns:
        try:
            df[col] = pd.to_datetime(df[col])
            return col
        except:
            continue
    raise ValueError("No date column found")

# Function to resample data
def resample_data(df, freq):
    resampled_df = df.resample(freq).mean().interpolate()
    return resampled_df

# Function to merge all sheets into daily data
def merge_to_daily(excel_file):
    # Read all sheets from the Excel file
    sheets = pd.read_excel(excel_file, sheet_name=None)
    daily_data = pd.DataFrame()

    for sheet_name, df in sheets.items():
        date_col = identify_date_format(df)
        df.set_index(date_col, inplace=True)
        df = df.resample('D').mean().interpolate()  # Convert to daily data

        if daily_data.empty:
            daily_data = df
        else:
            daily_data = daily_data.combine_first(df)

    return daily_data

# Function to normalize data
def normalize_data(df):
    # Get the first non-null value for each column to use as the base for normalization
    first_non_null = df.apply(lambda col: col.first_valid_index()).dropna()
    normalized_df = df.copy()

    for col in first_non_null.index:
        base_value = df.loc[first_non_null[col], col]
        normalized_df[col] = df[col] / base_value

    return normalized_df

# Specify the input file path
input_file = r'C:\Users\ardas\Downloads\Input\Bm.xlsx'

# Merge all sheets into daily data
daily_data = merge_to_daily(input_file)

# Define resampling frequencies
frequencies = {
    'daily': 'D',
    'weekly': 'W',
    'monthly': 'M',
    'quarterly': 'Q',
    'yearly': 'Y'
}

# Create a writer for the Excel file with raw data (using default engine)
output_file_raw = r'C:\Users\ardas\Downloads\resampled_timeseries_data.xlsx'
writer_raw = pd.ExcelWriter(output_file_raw, engine='openpyxl')

# Create a writer for the Excel file with normalized data
output_file_normalized = r'C:\Users\ardas\Downloads\normalized_timeseries_data.xlsx'
writer_normalized = pd.ExcelWriter(output_file_normalized, engine='openpyxl')

# Loop through frequencies, resample, normalize, and save to Excel
for freq_name, freq in frequencies.items():
    resampled_df = resample_data(daily_data, freq)
    resampled_df.to_excel(writer_raw, sheet_name=freq_name.capitalize())

    normalized_df = normalize_data(resampled_df)
    normalized_df.to_excel(writer_normalized, sheet_name=freq_name.capitalize())

# Save the Excel files
writer_raw.close()
writer_normalized.close()
